In [1]:
import os
from pathlib import Path
# codebase
from config import *
import luts
import slurm

Set the path to the directory containing the restacked dataset (this might have changed since last run):

In [4]:
restack_dir = Path(input() or "/beegfs/CMIP6/kmredilla/wrf_data/restacked")
print(restack_dir)

/beegfs/CMIP6/kmredilla/wrf_data/restacked


## Hourly data

Copy the hourly variables here.

In [6]:
varnames = [varname.lower() for varname in luts.varnames]

# use full spelling for each WRF group
aws_groups = [
    "GFDL-CM3_historical",
    "NCAR-CCSM4_historical",
    "GFDL-CM3_rcp85",
    "NCAR-CCSM4_rcp85",
    "ERA-Interim_historical"
]

sbatch_dir = slurm_dir.joinpath("copy_to_aws")
sbatch_dir.mkdir(exist_ok=True)
sbatch_fps = []
for varname in varnames:
    src_dir = restack_dir.joinpath("hourly", varname)
    
    for group in aws_groups:
        # write to .slurm script
        sbatch_fp = sbatch_dir.joinpath(f"copy_aws_{group}_{varname}.slurm")
        # filepath for slurm stdout
        sbatch_out_fp = sbatch_dir.joinpath(f"copy_aws_{group}_{varname}_%j.out")
        sbatch_head = slurm.make_sbatch_head(
            slurm_email, partition, conda_init_script
        )

        model, scenario = group.split("_")
        aws_dir = "s3://wrf-ak-ar5/hourly/{}/{}/{}".format(
            model, scenario, varname
        )
        
        args = {
            "sbatch_fp": sbatch_fp,
            "sbatch_out_fp": sbatch_out_fp,
            "s3_command": "sync",
            "src_dir": src_dir,
            "model": model,
            "aws_dir": aws_dir,
            "ncpus": 8,
            "sbatch_head": sbatch_head,
        }

        slurm.write_sbatch_copy_aws(**args)
        sbatch_fps.append(sbatch_fp)

    

Submit the jobs:

In [7]:
job_ids = [slurm.submit_sbatch(fp) for fp in sbatch_fps]

## Daily data

In [8]:
daily_sbatch_fps = []
for varname in luts.resample_varnames:
    varname = varname.lower()
    src_dir = restack_dir.joinpath("daily", varname)
    
    for group in aws_groups:
        # write to .slurm script
        sbatch_fp = sbatch_dir.joinpath(f"copy_aws_{group}_{varname}_daily.slurm")
        # filepath for slurm stdout
        sbatch_out_fp = sbatch_dir.joinpath(f"copy_aws_{group}_{varname}_daily_%j.out")
        sbatch_head = slurm.make_sbatch_head(
            slurm_email, partition, conda_init_script
        )

        model, scenario = group.split("_")
        aws_dir = "s3://wrf-ak-ar5/daily/{}/{}/{}".format(
            model, scenario, varname
        )
        
        args = {
            "sbatch_fp": sbatch_fp,
            "sbatch_out_fp": sbatch_out_fp,
            "s3_command": "sync",
            "src_dir": src_dir,
            "model": model,
            "aws_dir": aws_dir,
            "ncpus": 8,
            "sbatch_head": sbatch_head,
        }

        slurm.write_sbatch_copy_aws(**args)
        daily_sbatch_fps.append(sbatch_fp)


And submit:

In [9]:
job_ids = [slurm.submit_sbatch(fp) for fp in daily_sbatch_fps]